In [ ]:
import time
from multiprocessing import Pool

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from qpspin_mc.two_level_system import (
    SimulationParameters,
    SystemParameters,
    TwoLevelSystemSimulator,
)

# Magnetization as a function of $\Gamma$

In [ ]:
gammas = np.concatenate((np.linspace(0.001, 0.8, 4), np.linspace(1, 5, 8)))
beta = 2
h = 1

SEED = 5518
N_SAMPLES = 1_000
N_THINNING = 10
N_INIT_STEPS = 100 * N_THINNING

m_estimates = []
m_errs = []
l_samples = {}
acc_rates = []

for i, g in enumerate(gammas):
    system_parameters = SystemParameters(beta=beta, h=h, gamma=g)

    simulation_parameters = SimulationParameters(
        n_samples=N_SAMPLES,
        n_thinning=N_THINNING,
        n_init_steps=N_INIT_STEPS,
        seed=[i, SEED],
    )

    mc_result = TwoLevelSystemSimulator(system_parameters=system_parameters).run_mc(
        simulation_parameters
    )
    m_hat, m_sem = mc_result.estimate_magnetization()
    m_estimates.append(m_hat)
    m_errs.append(m_sem)
    l_samples[g] = [len(s) for s in mc_result.samples]
    acc_rates.append(mc_result.acceptance_rate())

m_estimates = np.array(m_estimates)
m_errs = np.array(m_errs)
acc_rates = np.array(acc_rates)

In [ ]:
gs = np.linspace(gammas.min(), gammas.max(), 100)
bs = np.sqrt(h**2 + gs**2)
m_exact = gs / bs * np.tanh(beta * bs)


fig, ax = plt.subplots()
ax.plot(gs, m_exact, label="Exact")
ax.errorbar(gammas, m_estimates, yerr=m_errs, fmt=".", label="MC estimate")
ax.plot(gammas, acc_rates, "kx", label="Acceptance rate")
ax.legend()

# Error and run time as a function of the number of samples

In [ ]:
gamma = 1
beta = 1
h = 1

NS = [10, 100, 200, 500, 1000, 2000, 4000, 6000, 8000, 10000, 20000, 50000, 100_000]

SEED = 5518
N_THINNING = 10
N_INIT_STEPS = 10 * N_THINNING


system_parameters = SystemParameters(beta=beta, h=h, gamma=gamma)


def run_simulation(args):
    i, ns = args
    simulation_parameters = SimulationParameters(
        n_samples=ns,
        n_thinning=N_THINNING,
        n_init_steps=N_INIT_STEPS,
        seed=[i, SEED],
    )

    st = time.perf_counter()
    mc_result = TwoLevelSystemSimulator(system_parameters=system_parameters).run_mc(
        simulation_parameters
    )
    et = time.perf_counter()
    m_hat, m_sem = mc_result.estimate_magnetization()
    return {"m_est": m_hat, "m_sem": m_sem, "time": et - st}


with Pool(processes=6) as pool:
    results = pd.DataFrame(pool.map(run_simulation, enumerate(NS)))

In [ ]:
m_exact = gamma / np.sqrt(h**2 + gamma**2) * np.tanh(beta * np.sqrt(h**2 + gamma**2))

fig, axes = plt.subplots(3, sharex=True, figsize=(6, 8))

axes[0].errorbar(
    NS, results["m_est"], yerr=results["m_sem"], fmt=".", label="MC estimate"
)
axes[0].axhline(m_exact, color="k", label="Exact")
axes[0].set_ylabel("Magnetization")
axes[0].legend()

axes[1].loglog(NS, results["m_sem"] / NS, "o")
axes[1].set_ylabel("MC relative error (SEM/mean)")
ax2 = axes[1].twinx()
ax2.loglog(NS, results["time"], "r.--", label="Time")
ax2.set_ylabel("Time (s)", color="r")
ax2.tick_params(axis="y")
axes[2].plot(NS, np.abs(results["m_est"] - m_exact) / results["m_sem"], "o")
axes[2].set_ylabel("# SEMs away from true value")
axes[2].set_xlim(xmin=0)